# SSD Object Detection для BCCD Dataset

Этот notebook реализует Single Shot MultiBox Detector (SSD300) для детекции клеток крови.

## Содержание:
1. Установка зависимостей и загрузка данных
2. Визуализация данных
3. Создание модели SSD300
4. Обучение модели
5. Оценка результатов
6. Inference на тестовых изображениях

## 1. Установка зависимостей и загрузка данных

In [ ]:
# Установка зависимостей
!pip install torch torchvision numpy matplotlib Pillow tqdm opencv-python

In [ ]:
# Клонирование датасета BCCD
!git clone https://github.com/Shenggan/BCCD_Dataset.git

In [ ]:
# Импорт необходимых библиотек
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import os
from tqdm import tqdm

# Импорт модулей проекта
import config
from model import SSD300
from dataset import BCCDDataset, create_dataloaders, TrainTransform, TestTransform
from utils import create_prior_boxes, MultiBoxLoss, parse_voc_annotation
from inference import detect, visualize_detection

print(f'PyTorch версия: {torch.__version__}')
print(f'Устройство: {config.DEVICE}')

## 2. Визуализация данных

In [ ]:
# Загрузка одного примера из датасета
sample_image_path = 'BCCD_Dataset/BCCD/JPEGImages/BloodImage_00000.jpg'
sample_annotation_path = 'BCCD_Dataset/BCCD/Annotations/BloodImage_00000.xml'

# Загрузка изображения
img = Image.open(sample_image_path)
boxes, labels, orig_size = parse_voc_annotation(sample_annotation_path)

print(f'Размер изображения: {orig_size}')
print(f'Количество объектов: {len(boxes)}')
print(f'Классы: {labels}')

In [ ]:
# Визуализация примера с аннотациями
def visualize_sample(image_path, annotation_path):
    img = Image.open(image_path)
    boxes, labels, (width, height) = parse_voc_annotation(annotation_path)
    
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(img)
    
    colors = {'WBC': 'red', 'RBC': 'blue', 'Platelets': 'green'}
    
    for box, label in zip(boxes, labels):
        xmin, ymin, xmax, ymax = box
        xmin *= width
        ymin *= height
        xmax *= width
        ymax *= height
        
        w = xmax - xmin
        h = ymax - ymin
        
        rect = patches.Rectangle(
            (xmin, ymin), w, h,
            linewidth=2, edgecolor=colors[label], facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(xmin, ymin-5, label, color=colors[label], fontsize=12)
    
    ax.axis('off')
    plt.title('Пример аннотированного изображения BCCD')
    plt.show()

visualize_sample(sample_image_path, sample_annotation_path)

In [ ]:
# Статистика по датасету
images_dir = config.TRAIN_IMAGES_DIR
annotations_dir = config.TRAIN_ANNOTATIONS_DIR

class_counts = {'WBC': 0, 'RBC': 0, 'Platelets': 0}
total_images = 0

for ann_file in os.listdir(annotations_dir):
    if ann_file.endswith('.xml'):
        ann_path = os.path.join(annotations_dir, ann_file)
        _, labels, _ = parse_voc_annotation(ann_path)
        for label in labels:
            class_counts[label] += 1
        total_images += 1

print(f'\nСтатистика датасета BCCD:')
print(f'Всего изображений: {total_images}')
print(f'\nКоличество объектов по классам:')
for cls, count in class_counts.items():
    print(f'  {cls}: {count}')
print(f'\nВсего объектов: {sum(class_counts.values())}')

## 3. Создание модели SSD300

In [ ]:
# Создание модели
device = config.DEVICE
model = SSD300(n_classes=config.NUM_CLASSES)
model = model.to(device)

# Подсчет параметров
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Модель SSD300 создана')
print(f'Всего параметров: {total_params:,}')
print(f'Обучаемых параметров: {trainable_params:,}')

In [ ]:
# Создание prior boxes
priors = create_prior_boxes()
print(f'Создано prior boxes: {priors.size(0)}')
print(f'Размер prior boxes: {priors.shape}')

In [ ]:
# Проверка forward pass
dummy_input = torch.randn(1, 3, 300, 300).to(device)
with torch.no_grad():
    locs, scores = model(dummy_input)
    
print(f'Output shapes:')
print(f'  Localization predictions: {locs.shape}')
print(f'  Class scores: {scores.shape}')

## 4. Обучение модели

In [ ]:
# Создание DataLoaders
print('Создание DataLoaders...')
train_loader, val_loader = create_dataloaders(train_split=0.8)

print(f'Train batches: {len(train_loader)}')
print(f'Val batches: {len(val_loader)}')

In [ ]:
# Настройка обучения
priors = priors.to(device)
criterion = MultiBoxLoss(priors, alpha=config.ALPHA)

# Optimizer с разными learning rates для bias и weights
biases = []
not_biases = []
for param_name, param in model.named_parameters():
    if param.requires_grad:
        if param_name.endswith('.bias'):
            biases.append(param)
        else:
            not_biases.append(param)

optimizer = optim.SGD(
    [
        {'params': biases, 'lr': 2 * config.LEARNING_RATE},
        {'params': not_biases}
    ],
    lr=config.LEARNING_RATE,
    momentum=config.MOMENTUM,
    weight_decay=config.WEIGHT_DECAY
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer,
    milestones=config.LR_DECAY_EPOCHS,
    gamma=config.LR_DECAY_FACTOR
)

print('Optimizer и scheduler созданы')

In [ ]:
# Функции для обучения
def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch):
    model.train()
    running_loss = 0.0
    
    pbar = tqdm(dataloader, desc=f'Epoch {epoch+1}')
    
    for batch_idx, (images, boxes, labels) in enumerate(pbar):
        images = images.to(device)
        boxes = [b.to(device) for b in boxes]
        labels = [l.to(device) for l in labels]
        
        # Forward pass
        predicted_locs, predicted_scores = model(images)
        
        # Loss
        loss = criterion(predicted_locs, predicted_scores, boxes, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
        
        optimizer.step()
        
        running_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 
                         'avg_loss': f'{running_loss/(batch_idx+1):.4f}'})
    
    return running_loss / len(dataloader)

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    
    with torch.no_grad():
        for images, boxes, labels in tqdm(dataloader, desc='Validation'):
            images = images.to(device)
            boxes = [b.to(device) for b in boxes]
            labels = [l.to(device) for l in labels]
            
            predicted_locs, predicted_scores = model(images)
            loss = criterion(predicted_locs, predicted_scores, boxes, labels)
            
            running_loss += loss.item()
    
    return running_loss / len(dataloader)

In [ ]:
# Обучение (можно уменьшить NUM_EPOCHS для быстрого теста)
NUM_EPOCHS = 50  # Для демо используем 50 эпох

os.makedirs(config.CHECKPOINT_DIR, exist_ok=True)

train_losses = []
val_losses = []
best_val_loss = float('inf')

print('Начало обучения...')

for epoch in range(NUM_EPOCHS):
    # Train
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch)
    train_losses.append(train_loss)
    
    # Validate
    val_loss = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    
    # LR scheduler step
    scheduler.step()
    
    print(f'\nEpoch {epoch+1}/{NUM_EPOCHS}:')
    print(f'  Train Loss: {train_loss:.4f}')
    print(f'  Val Loss: {val_loss:.4f}')
    print(f'  LR: {optimizer.param_groups[0]["lr"]:.6f}\n')
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_path = os.path.join(config.CHECKPOINT_DIR, 'best_model.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
        }, best_model_path)
        print(f'Лучшая модель сохранена: {best_model_path}')

print('Обучение завершено!')

## 5. Оценка результатов

In [ ]:
# График loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss (Log Scale)')
plt.yscale('log')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f'Лучший Val Loss: {best_val_loss:.4f}')

## 6. Inference на тестовых изображениях

In [ ]:
# Загрузка лучшей модели
checkpoint = torch.load(os.path.join(config.CHECKPOINT_DIR, 'best_model.pth'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
print('Лучшая модель загружена для inference')

In [ ]:
# Inference на нескольких тестовых изображениях
test_images = [
    'BCCD_Dataset/BCCD/JPEGImages/BloodImage_00001.jpg',
    'BCCD_Dataset/BCCD/JPEGImages/BloodImage_00010.jpg',
    'BCCD_Dataset/BCCD/JPEGImages/BloodImage_00050.jpg',
]

for img_path in test_images:
    if os.path.exists(img_path):
        print(f'\nОбработка: {img_path}')
        boxes, labels, scores, original_image = detect(
            img_path, model, device,
            min_score=0.3,
            max_overlap=0.45
        )
        
        print(f'Найдено объектов: {boxes.size(0)}')
        for i in range(boxes.size(0)):
            label = labels[i].item()
            if label > 0:
                class_name = config.IDX_TO_CLASS[label]
                score = scores[i].item()
                print(f'  {class_name}: {score:.3f}')
        
        visualize_detection(original_image, boxes, labels, scores, show=True)
    else:
        print(f'Файл не найден: {img_path}')

In [ ]:
# Сохранение финальной модели
final_model_path = 'ssd300_bccd_final.pth'
torch.save(model.state_dict(), final_model_path)
print(f'Финальная модель сохранена: {final_model_path}')

## Заключение

В этом notebook мы:
1. Загрузили и исследовали датасет BCCD
2. Реализовали архитектуру SSD300
3. Обучили модель для детекции клеток крови
4. Протестировали модель на новых изображениях

### Дальнейшие улучшения:
- Увеличить количество эпох обучения
- Добавить более агрессивную аугментацию данных
- Использовать другие backbone (ResNet, MobileNet)
- Реализовать вычисление mAP метрики
- Экспериментировать с гиперпараметрами